In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import StorageContext
from llama_index.core.query_pipeline import QueryPipeline
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.agent.openai import OpenAIAgent
#from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
import os
from tqdm.notebook import tqdm
import pickle
from pathlib import Path

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Retrieve the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [2]:
def read_folder(path):
    # Walk through all files in the given path and return a list of documents
    documents = []
    all_files = []
    for root, dirs, files in os.walk(path):
        all_files.append(files)
        for file in files:
            document_path = os.path.join(root, file)
            documents.append(SimpleDirectoryReader(input_files=[document_path]).load_data())
            

    indexes = []
    for i in documents:
        indexes.append(VectorStoreIndex.from_documents(documents=i))
        
    return indexes, files

In [3]:
indexes, files = read_folder('data/')

llm = OpenAI(model="gpt-3.5-turbo-0613")
#print(files)

#tools = create_engines_tools(indexes)

tools = [QueryEngineTool.from_defaults(
    indexes[i].as_query_engine(), name=f"engine_{i}", description=f"Provides information about {files[i]}"
) for i in range(len(indexes))]


agent = OpenAIAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True
)

Failed to load file data/customer data - preffered car price.csv with error: 'utf-8' codec can't decode byte 0xc5 in position 8694: invalid continuation byte. Skipping...


In [4]:
agent.chat("What are the characteristics of Volkswagen ID.3 Pro")

Added user message to memory: What are the characteristics of Volkswagen ID.3 Pro
=== Calling Function ===
Calling function: engine_0 with args: {
  "input": "Volkswagen ID.3 Pro"
}
Got output: The Volkswagen ID.3 Pro is a hatchback electric car with a range of 340 miles, rear-wheel drive, Type 2 CCS charging capability, falls under category C, seats 5 people, and is priced at 35,575.



AgentChatResponse(response='The Volkswagen ID.3 Pro is a hatchback electric car with the following characteristics:\n\n- Range: 340 miles\n- Drive: Rear-wheel drive\n- Charging Capability: Type 2 CCS\n- Category: C\n- Seating Capacity: 5 people\n- Price: $35,575', sources=[ToolOutput(content='The Volkswagen ID.3 Pro is a hatchback electric car with a range of 340 miles, rear-wheel drive, Type 2 CCS charging capability, falls under category C, seats 5 people, and is priced at 35,575.', tool_name='engine_0', raw_input={'input': 'Volkswagen ID.3 Pro'}, raw_output=Response(response='The Volkswagen ID.3 Pro is a hatchback electric car with a range of 340 miles, rear-wheel drive, Type 2 CCS charging capability, falls under category C, seats 5 people, and is priced at 35,575.', source_nodes=[NodeWithScore(node=TextNode(id_='5db74dc4-2ef8-40cc-8a21-fbeb1dcdf2d0', embedding=None, metadata={'file_path': 'data/ElectricCarData_Clean.csv', 'file_name': 'ElectricCarData_Clean.csv', 'file_type': 'tex